# Thermal explosion

We consider the following system :

$$
d_t \theta = F_k \, e^{\theta} - \theta
$$

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import Label

from ipywidgets import interact, FloatSlider

from mylib.model import thermal_explosion_model
import mylib.continuation as continuation

import warnings
warnings.filterwarnings('ignore')

output_notebook(hide_banner=True)

In [2]:
def plot_sol():
    
    fk_lim = 1/np.exp(1)  
    
    fk = fk_lim

    tem = thermal_explosion_model(fk)
    fcn = tem.fcn
    jac = tem.jac
    
    tini = 0. 
    tend = 10.
    
    yini = (0.,)
    
    tol = 1.e-10
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_y = fig_sol.line(sol.t, sol.y[0], line_width=2)
    fig_sol.legend.location = "top_left"
    
    s_ini = 0.
    s_end = 10.
    ns = 201 
    fk_ini = 0.
    yeq_ini = (0.,)
    fkeq, yeq = continuation.pseudo_arclength_continuation(s_ini, s_end, ns, fk_ini, yeq_ini, thermal_explosion_model)

    yeq1 = np.array([0.])
    for it_newton in range(100):
        yeq1 += np.linalg.solve(jac(0, yeq1), -fcn(0, yeq1))
        if (np.linalg.norm(fcn(0, yeq1)) < 1.e-7):
            break

    yeq2 = np.array([9.5])
    for it_newton in range(100):
        yeq2 += np.linalg.solve(jac(0, yeq2), -fcn(0, yeq2))
        if (np.linalg.norm(fcn(0, yeq2)) < 1.e-7):
            break
    
    fig_eq = figure(width=475, height=300, title="Equilibrium points")
    fig_eq.line(fkeq, yeq[0])
    plt_eq = fig_eq.circle(np.array([fk,fk]), np.array([yeq1[0], yeq2[0]]), line_width=2, size=10)
    
    fig_eig = figure(x_range=(0, 0.4), y_range=(-2, 7), width=475, height=300, 
                    title="Eigen value for each branch at equilibrium points")
    plt_eig_2 = fig_eig.circle((fk,), (yeq2[0],), line_width=2, size=10, color="red", legend="unstable point")
    plt_eig_1 = fig_eig.circle((fk,), (yeq1[0],), line_width=2, size=10, color="green", legend="stable point")

    show(column(fig_sol, row(fig_eq, fig_eig)), notebook_handle=True)
    
    def update(fk):
        
        tem = thermal_explosion_model(fk)
        fcn = tem.fcn
        jac = tem.jac
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        
        yeq1 = np.array([0.])
        for it_newton in range(100):
            yeq1 += np.linalg.solve(jac(0, yeq1), -fcn(0, yeq1))
            if (np.linalg.norm(fcn(0, yeq1)) < 1.e-7):
                break

        yeq2 = np.array([9.5])
        for it_newton in range(100):
            yeq2 += np.linalg.solve(jac(0, yeq2), -fcn(0, yeq2))
            if (np.linalg.norm(fcn(0, yeq2)) < 1.e-7):
                break

        plt_y.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_eq.data_source.data = dict(x=np.array([fk,fk]), y=np.array([yeq1[0], yeq2[0]]))
        plt_eig_2.data_source.data = dict(x=(fk,), y=(yeq2[0]-1,))
        plt_eig_1.data_source.data = dict(x=(fk,), y=(yeq1[0]-1,))

        push_notebook()

    interact(update, fk=FloatSlider(min=0.01 , max=fk_lim, value=fk_lim, step=(fk_lim-0.01)/100, 
                                    continuous_update=False));

plot_sol()

[0.03535534 0.03535534]
[ 0.00063638 -0.00063638]
[-2.06069218e-07  2.06069218e-07]
[-2.16214455e-14  2.16214455e-14]
[0.03661487 0.03404924]
[ 0.00064386 -0.00069238]
[-2.22759761e-07  2.39544832e-07]
[-2.66790031e-14  2.86892806e-14]
[0.03788505 0.0326301 ]
[ 0.00064481 -0.00074866]
[-2.36102554e-07  2.74125975e-07]
[-3.16696368e-14  3.67592790e-14]
[0.03915238 0.03109808]
[ 0.00063822 -0.00080351]
[-2.44441218e-07  3.07750671e-07]
[-3.58728786e-14  4.51861464e-14]
[0.04040157 0.02945698]
[ 0.00062335 -0.00085496]
[-2.46260183e-07  3.37756882e-07]
[-3.84435315e-14  5.27388051e-14]
[0.04161606 0.02771468]
[ 0.0005999 -0.0009008]
[-2.40508153e-07  3.61144501e-07]
[-3.86863802e-14  5.81035781e-14]
[0.04277902 0.0258835 ]
[ 0.00056806 -0.00093885]
[-2.26911609e-07  3.75028712e-07]
[-3.62472369e-14  5.98809005e-14]
[0.04387431 0.02398009]
[ 0.00052862 -0.00096717]
[-2.06168642e-07  3.77209105e-07]
[-3.13765520e-14  5.73780538e-14]
[0.04488772 0.02202482]
[ 0.00048294 -0.00098426]
[-1.7992

[1.65708852e-09 1.33352105e-06]
[-1.76218526e-16 -2.20984601e-15]
[ 4.99999646e-02 -5.94837530e-05]
[1.52025998e-09 1.27787684e-06]
[ 5.11167225e-16 -1.94278745e-15]
[ 4.99999676e-02 -5.69460414e-05]
[1.39459603e-09 1.22448834e-06]
[-4.04400877e-16 -1.70755367e-15]
[ 4.99999703e-02 -5.45143746e-05]
[1.27920000e-09 1.17326788e-06]
[-7.44098405e-16 -1.50081044e-15]
[ 4.99999728e-02 -5.21844450e-05]
[1.17324279e-09 1.12413097e-06]
[-4.99486548e-17 -1.31875531e-15]
[ 4.9999975e-02 -4.9952113e-05]
[1.07596587e-09 1.07699621e-06]
[-1.19118979e-16 -1.15883279e-15]
[ 4.99999771e-02 -4.78134006e-05]
[9.8666358e-10 1.0317852e-06]
[ 3.80665930e-16 -1.01792438e-15]
[ 4.99999791e-02 -4.57644860e-05]
[9.04692964e-10 9.88422401e-07]
[-2.43679750e-16 -8.94101065e-16]
[ 4.99999808e-02 -4.38016972e-05]
[8.29460788e-10 9.46835074e-07]
[-1.60282688e-16 -7.85433479e-16]
[ 4.99999824e-02 -4.19215070e-05]
[7.60416484e-10 9.06953182e-07]
[ 7.55761423e-16 -6.89714937e-16]
[ 4.99999839e-02 -4.01205271e-05]
[6.9

interactive(children=(FloatSlider(value=0.36787944117144233, continuous_update=False, description='fk', max=0.…

* by continuation algo find the left plot, 蓝色的点是fold point
* 上面的是unstable下面的stable，数值解并不知道哪一个是stable
* arc length function可以让下面的点，continuation method往上走，走到fold之后还可以继续向上
* 要么就不知道啥时候该转折，可能就直接停止了

* 两个valeur propre是为了两个configuration，有一个stable有一个不行

In [3]:
def show_pseudo_arclength_continuation():
    
    s_ini = 0
    s_end = 10 
    ns = 101
    
    fk_ini = 0.
    theta_eq_ini = (0., )
    
    fk, theta_eq = continuation.pseudo_arclength_continuation(s_ini, s_end, ns, fk_ini, theta_eq_ini, 
                                                              thermal_explosion_model)
    #theta_eq=np.linspace(0,10,ns-1)            
    return theta_eq
    theta_stable   = theta_eq[np.where(theta_eq < 1)]
    theta_unstable = theta_eq[np.where(theta_eq > 1)]
    fk_stable   = fk[np.where(theta_eq < 1)[1]]
    fk_unstable   = fk[np.where(theta_eq > 1)[1]]

    theta_lim = (theta_unstable[0] + theta_stable[-1])/2  
    fk_lim = (fk_unstable[0] + fk_stable[-1])/2

    fig_branch = figure(plot_height=500, plot_width=950, x_range=(0.0,0.425), y_range=(-1,10))    
    fig_branch.x(fk_stable, theta_stable, color="green", legend="stable branch")
    fig_branch.x(fk_unstable, theta_unstable, color="red", legend="unstable branch")
    fig_branch.line((0,0.425), (theta_lim, theta_lim), line_dash="dashed")
    fig_branch.line((1/np.exp(1),1/np.exp(1)), (-1, 10), line_dash="dashed")
    fig_branch.add_layout(Label(x=fk_lim+0.005, y=theta_lim+0.05, text="Limit point"))
    fig_branch.legend.location = "top_left"

    show(fig_branch)

a=show_pseudo_arclength_continuation()

[0.07071068 0.07071068]
[ 0.00259391 -0.00259391]
[-3.48278858e-06  3.48278858e-06]
[-6.29524909e-12  6.29524909e-12]
[0.07577352 0.06525621]
[ 0.00261894 -0.00304103]
[-3.96661859e-06  4.60591652e-06]
[-9.12371038e-12  1.05941955e-11]
[0.08081044 0.05890393]
[ 0.0025193  -0.00345623]
[-4.09616017e-06  5.61953129e-06]
[-1.08566165e-11  1.48942165e-11]
[0.08556887 0.05174909]
[ 0.00228161 -0.00377271]
[-3.72262190e-06  6.15548052e-06]
[-9.93303969e-12  1.64246421e-11]
[0.08978846 0.04402309]
[ 0.00192613 -0.00392848]
[-2.90349278e-06  5.92189546e-06]
[-6.61075937e-12  1.34830860e-11]
[0.09326784 0.03607091]
[ 0.00150539 -0.00389246]
[-1.91091641e-06  4.94101868e-06]
[-3.08390932e-12  7.97388777e-12]
[0.09591805 0.02827944]
[ 0.0010844  -0.00367808]
[-1.05086615e-06  3.56432158e-06]
[-9.87961907e-13  3.35091428e-12]
[0.09777244 0.02098929]
[ 0.00071571 -0.00333392]
[-4.77770286e-07  2.22555226e-06]
[-2.13051936e-13  9.92505855e-13]
[0.09895281 0.01443407]
[ 0.00042603 -0.00292064]
[-1.74

* 两种bifurcation，sensibilite infini, 在limit point那里，continuation arc使用了curiline，不只是用axe而是courb
* 只用数值解不可能到红色的那边，必须用continuation的方法来做才能解出上面的

In [4]:
a[0]

array([0.        , 0.0733011 , 0.1516896 , 0.23501524, 0.32286199,
       0.41457367, 0.50934499, 0.60634639, 0.70483406, 0.80421272,
       0.90404935, 1.00405476, 1.10405211, 1.20394507, 1.30369122,
       1.40328213, 1.50272934, 1.60205489, 1.70128525, 1.80044753,
       1.89956723, 1.99866704, 2.09776632, 2.19688095, 2.29602349,
       2.39520346, 2.49442767, 2.59370062, 2.69302487, 2.7924014 ,
       2.89182994, 2.99130923, 3.09083728, 3.19041161, 3.29002936,
       3.3896875 , 3.48938289, 3.5891124 , 3.68887298, 3.78866168,
       3.8884757 , 3.98831242, 4.08816942, 4.18804446, 4.28793548,
       4.38784062, 4.48775821, 4.58768674, 4.68762486, 4.78757136,
       4.88752517, 4.98748536, 5.08745108, 5.1874216 , 5.28739629,
       5.38737457, 5.48735596, 5.58734002, 5.6873264 , 5.78731475,
       5.88730481, 5.98729633, 6.08728911, 6.18728295, 6.28727772,
       6.38727327, 6.48726948, 6.58726627, 6.68726354, 6.78726123,
       6.88725927, 6.98725761, 7.08725621, 7.18725502, 7.28725

In [5]:
10/a[0][1]


136.42359380664593